In [2]:
# NOTAS:
# Codigo a ejecutar en un entorno local. Probado en Jupyter notebook
#!pip install selenium  # <-----INSTALAR LIBRERIA
# Instalar el Chrome Driver para navegador automático

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import numpy as np
import time

# Web scraping sobre NASDAQ

In [2]:
"""
En la pagina oficial de Nasdaq se encuentra el listado total de
empresas que cotizan dentro de su mercado.
EL siguiente codigo automatiza la obtención de ese archivo.
"""

# Configura el controlador de Selenium 
driver = webdriver.Chrome()

try:
    # Navegar a la página
    pagina_nasdaq = 'https://www.nasdaq.com/market-activity/stocks/screener'
    #xpath del boton para bajar el csv actualizado
    xpath_completo = '/html/body/div[2]/div/main/div[2]/article/div[3]/div[1]/div/div/div[3]/div[2]/div[2]'
    
    driver.get(pagina_nasdaq)

    # Esperar hasta que el botón de descarga esté presente
    wait = WebDriverWait(driver, 10)
    download_button = wait.until(EC.element_to_be_clickable((By.XPATH,
        xpath_completo)))
    # Hacer clic en el botón de descarga
    download_button.click()
    # Esperar un tiempo para que se complete la descarga
    time.sleep(25)  # Ajusta este tiempo según sea necesario para asegurar la descarga

finally:
    # Cerrar el navegador
     driver.quit()
# 
"""
    El archivo se baja por defecto en la carpeta de descarga del sistema.
    una vez bajado hay que pegarlo en la carpeta donde se encuentre 
    el archivo de este codigo, o  leerlo desde la carpeta descarga.
    Renombrarlo a 'nasdaq_screener.csv', ya que cada vez que baja lo hace
    con nombre distinto.
    Si el codigo falla, por lo general es que ha cambiado el x_path del boton, 
    o la pagina colocó algun tipo de restricción a navegadores automáticos.
    Solucion: entrar a la pagina y bajarlo manualmente.
    
"""

"\n    El archivo se baja por defecto en la carpeta de descarga del sistema.\n    una vez bajado hay que pegarlo en la carpeta donde se encuentre \n    el archivo de este codigo, o  leerlo desde la carpeta descarga.\n    Renombrarlo a 'nasdaq_screener.csv', ya que cada vez que baja lo hace\n    con nombre distinto.\n    Si el codigo falla, por lo general es que ha cambiado el x_path del boton, \n    o la pagina colocó algun tipo de restricción a navegadores automáticos.\n    Solucion: entrar a la pagina y bajarlo manualmente.\n    \n"

In [3]:
# leer el archivo
nasdaq_total = pd.read_csv('nasdaq_screener.csv', sep=',')

nasdaq_total.shape

#de este archivo solo nos interesa las columnas ['Symbol', 'Name', 'Market Cap','Country', 'IPO Year', 'Sector', 'Industry']
nasdaq_total.columns

Index(['Symbol', 'Name', 'Last Sale', 'Net Change', '% Change', 'Market Cap',
       'Country', 'IPO Year', 'Volume', 'Sector', 'Industry'],
      dtype='object')

In [4]:
nasdaq_total2 = nasdaq_total[['Symbol', 'Name', 'Market Cap','Country', 'IPO Year', 'Sector', 'Industry']].copy()
nasdaq_total2 = nasdaq_total2.rename(columns={'Symbol': 'symbol', 'Market Cap': 'Market Cap2'})

nasdaq_total2 

,symbol,Name,Market Cap2,Country,IPO Year,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,4.415171e+10,United States,1999.0,Industrials,Electrical Products
1,AA,Alcoa Corporation Common Stock,7.476865e+09,United States,2016.0,Industrials,Aluminum
2,AACG,ATA Creativity Global American Depositary Shares,2.720331e+07,China,2008.0,Real Estate,Other Consumer Services
3,AACI,Armada Acquisition Corp. I Common Stock,0.000000e+00,United States,2021.0,Finance,Blank Checks
4,AACIW,Armada Acquisition Corp. I Warrant,0.000000e+00,United States,2021.0,Finance,Blank Checks
...,...,...,...,...,...,...,...
7116,ZVRA,Zevra Therapeutics Inc. Common Stock,1.950373e+08,United States,NaN,Health Care,Biotechnology: Pharmaceutical Preparations
7117,ZVSA,ZyVersa Therapeutics Inc. Common Stock,3.598402e+06,United States,2022.0,Health Care,Biotechnology: Pharmaceutical Preparations
7118,ZWS,Zurn Elkay Water Solutions Corporation Common ...,5.517664e+09,United States,2012.0,Industrials,Industrial Machinery/Components
7119,ZYME,Zymeworks Inc. Common Stock,6.165466e+08,United States,NaN,Health Care,Biotechnology: Pharmaceutical Preparations


# Web Scraping sobre Finviz

In [5]:
# Lista de más de 7000 simbolos
nasdaq7000 = nasdaq_total2['symbol'].to_list()
print(f'cantidad de simbolos:{len(nasdaq7000)}')

cantidad de simbolos:7121



En el repositorio existe un archivo .txt con las empresas del Nasdaq 100


In [6]:
# ARCHIVO DE PRUEBA 

# Leer el archivo 'nasdaq100.txt'
with open('nasdaq100.txt', 'r') as f:
    nasdaq100_data = f.read()

# crear lista con los simbolos del nasdaq 100
nasdaq100 = nasdaq100_data.splitlines()
len(nasdaq100)

11

In [8]:
# Diccionario para almacenar DataFrames
dataframes = {}
simbolos_no_encontrados = []  # Inicializar la lista para símbolos no encontrados

# Configura el controlador de Selenium 
driver = webdriver.Chrome()


#for simbolo in nasdaq7000: #iterar sobre la lista completa (puede tardar hasta 10 hs) 

for simbolo in nasdaq100: # archivo de prueba
    try:
    # URL con el símbolo actual
        url = f'https://finviz.com/quote.ashx?t={simbolo}&p=d'
    
        # Abre la URL
        driver.get(url)
        time.sleep(2)
    
        try:
            # Intenta encontrar la tabla utilizando el primer XPath
            table_xpath = '/html/body/div[3]/div[3]/div[4]/table/tbody/tr/td/div/table[1]/tbody/tr/td/div[2]/table/tbody'
            table = driver.find_element(By.XPATH, table_xpath)
        except Exception:
            # Si falla, intenta encontrar la tabla utilizando el segundo XPath
            table_xpath = '/html/body/div[2]/div[3]/div[4]/table/tbody/tr/td/div/table[1]/tbody/tr/td/div[2]/table/tbody'
            table = driver.find_element(By.XPATH, table_xpath)
    
        # Extrae los datos de la tabla y guarda en un DataFrame
        data = []
        rows = table.find_elements(By.TAG_NAME, 'tr')
        for row in rows:
            cols = row.find_elements(By.TAG_NAME, 'td')
            cols = [col.text for col in cols]
            data.append(cols)
        df = pd.DataFrame(data[1:], columns=data[0])
    
        # Almacena el DataFrame con el nombre del símbolo como clave
        dataframes[simbolo] = df
            
    except Exception as e:
        # Imprime el símbolo no encontrado y pasa al siguiente
        print(f"Símbolo no encontrado: {simbolo}\n")
        #print(f"Exception: {e}")
        simbolos_no_encontrados.append(simbolo)
        continue

# Cierra el navegador controlado por Selenium
driver.quit()

# Imprime el diccionario
#print(dataframes)
#guardar el diccionario en un pickle
with open('dataframes.pkl', 'wb') as file:
    pd.to_pickle(dataframes, file)

Símbolo no encontrado: WWRR

Símbolo no encontrado: GDGG



In [11]:
simbolos_no_encontrados

['WWRR', 'GDGG']

In [83]:
dataframes.keys()

dict_keys(['AAPL', 'ABNB', 'ADBE', 'ADI', 'ADP', 'TSLA', 'MSFT', 'XOM', 'MELI'])

In [13]:
# El diccionario contiene los dataframes en el formato mostrado en la web:
dataframes['AAPL']

,Index,"DJIA, NDX, S&P 500",P/E,29.54,EPS (ttm),6.43,Insider Own,0.09%,Shs Outstand,15.55B,Perf Week,0.06%
0,Market Cap,2913.17B,Forward P/E,26.40,EPS next Y,7.20,Insider Trans,-3.69%,Shs Float,15.32B,Perf Month,11.83%
1,Income,100.39B,PEG,2.81,EPS next Q,1.33,Inst Own,61.96%,Short Float,0.65%,Perf Quarter,4.87%
2,Sales,381.62B,P/S,7.63,EPS this Y,7.52%,Inst Trans,-0.48%,Short Ratio,1.55,Perf Half Y,-0.70%
3,Book/sh,4.84,P/B,39.27,EPS next Y,9.18%,ROA,29.99%,Short Interest,99.29M,Perf Year,10.56%
4,Cash/sh,4.38,P/C,43.38,EPS next 5Y,10.50%,ROE,147.25%,52W Range,164.07 - 199.62,Perf YTD,-1.32%
5,Dividend Est.,0.99 (0.52%),P/FCF,28.58,EPS past 5Y,15.55%,ROI,60.47%,52W High,-4.83%,Beta,1.24
6,Dividend TTM,0.97 (0.51%),Quick Ratio,0.99,Sales past 5Y,8.34%,Gross Margin,45.59%,52W Low,15.79%,ATR (14),3.30
7,Dividend Ex-Date,"May 10, 2024",Current Ratio,1.04,EPS Y/Y TTM,9.23%,Oper. Margin,30.98%,RSI (14),63.70,Volatility,1.50% 1.58%
8,Employees,161000,Debt/Eq,1.41,Sales Y/Y TTM,-0.90%,Profit Margin,26.31%,Recom,2.02,Target Price,203.01
9,Option/Short,Yes / Yes,LT Debt/Eq,1.24,EPS Q/Q,0.25%,Payout,15.32%,Rel Volume,0.57,Prev Close,186.88


In [84]:
def reestruc_df(df,symbol):
    # Los heads del dataframe deben manejarse como parte de los datos, 
    linea = df.columns.to_list()
    df.loc[-1]=linea
    df.index =df.index+1
    df=df.sort_index()
    columnas=list(range(df.shape[1]))
    df.columns =columnas
    
    # Crear una lista vacía para almacenar los DataFrames resultantes
    dataframes = []

    # Dividir en 6 DataFrames
    for i in range(0, 12, 2): #en col 0,2,4,6,8,10 están los heads
        # Extraer las columnas para cada DataFrame
        df_temp = df.iloc[:, i:i+2]
        # Asignar nombre al DataFrame
        df_temp.name = f'df{i}'  # Por ejemplo, df1, df2, df3, etc.
        #
        # Agregar el DataFrame a la lista
        dataframes.append(df_temp)

    # Crear un df vacío para almacenar los DataFrames transpuestos
    df_final= pd.DataFrame()

    # Transponer y concatenar los DataFrames
    for df_temp in dataframes:
        # Transponer el DataFrame
        df_temp2 = df_temp.T
        df_temp2.columns =df_temp2.iloc[0]
        df_temp2=df_temp2[1:]
        df_temp2.reset_index(drop=True,inplace= True)
        # Agregar el DataFrame transpuesto a la lista
        df_final = pd.concat([df_final,df_temp2],axis = 1, ignore_index=True)
    # Agregar la columna 'symbol' en la posición 0 con el valor 'tsla'
    df_final.insert(0,'symbol', symbol)
    # Mostr ar el DataFrame concatenado
    
    return(df_final)



In [85]:
columnas = ['symbol', 'Index', 'Market Cap', 'Income', 'Sales', 'Book/sh', 'Cash/sh', 'Dividend Est.', 'Dividend TTM', 'Dividend Ex-Date', 'Employees', 'Option/Short', 'Sales Surprise', 'SMA20', 'P/E', 'Forward P/E', 'PEG', 'P/S', 'P/B', 'P/C', 'P/FCF', 'Quick Ratio', 'Current Ratio', 'Debt/Eq', 'LT Debt/Eq', 'EPS Surprise', 'SMA50', 'EPS (ttm)', 'EPS next Y', 'EPS next Q', 'EPS this Y','EPS next Y (%)' ,'EPS next 5Y', 'EPS past 5Y', 'Sales past 5Y', 'EPS Y/Y TTM', 'Sales Y/Y TTM', 'EPS Q/Q', 'Sales Q/Q', 'SMA200', 'Insider Own', 'Insider Trans', 'Inst Own', 'Inst Trans', 'ROA', 'ROE', 'ROI', 'Gross Margin', 'Oper. Margin', 'Profit Margin', 'Payout', 'Earnings', 'Trades', 'Shs Outstand', 'Shs Float', 'Short Float', 'Short Ratio', 'Short Interest', '52W Range', '52W High', '52W Low', 'RSI (14)', 'Recom', 'Rel Volume', 'Avg Volume', 'Volume', 'Perf Week', 'Perf Month', 'Perf Quarter', 'Perf Half Y', 'Perf Year', 'Perf YTD', 'Beta', 'ATR (14)', 'Volatility', 'Target Price', 'Prev Close', 'Price', 'Change']

In [86]:

# Crea un DataFrame vacío para almacenar los DataFrames reestructurados
data_nasdaq = pd.DataFrame()
# Crea una lista vacía para almacenar los DataFrames reestructurados
data_nasdaq_list = []

# Itera sobre el diccionario 'dataframes'
for simbolo, df in dataframes.items():
    # Reestructura el DataFrame actual utilizando la función 'reestruc_df'
    df1 = df.copy()
    df_reestructurado= reestruc_df(df1, simbolo)
    data_nasdaq = pd.concat([data_nasdaq,df_reestructurado], axis=0, ignore_index=True)
data_nasdaq.columns=columnas
# Concatena todos los DataFrames en la lista en un solo DataFrame
data_nasdaq

# Resultado:


,symbol,Index,Market Cap,Income,Sales,Book/sh,Cash/sh,Dividend Est.,Dividend TTM,Dividend Ex-Date,...,Perf Half Y,Perf Year,Perf YTD,Beta,ATR (14),Volatility,Target Price,Prev Close,Price,Change
0,AAPL,"DJIA, NDX, S&P 500",2913.17B,100.39B,381.62B,4.84,4.38,0.99 (0.52%),0.97 (0.51%),"May 10, 2024",...,-0.70%,10.56%,-1.32%,1.24,3.30,1.50% 1.58%,203.01,186.88,189.98,1.66%
1,ABNB,"NDX, S&P 500",91.72B,4.94B,10.24B,12.42,17.53,-,-,-,...,11.96%,36.67%,6.12%,1.18,3.72,1.55% 2.13%,151.11,141.04,144.47,2.43%
2,ADBE,"NDX, S&P 500",212.99B,4.80B,19.92B,34.13,15.22,-,-,"Mar 24, 2005",...,-23.28%,29.98%,-20.31%,1.29,10.18,2.03% 2.08%,619.77,483.31,475.43,-1.63%
3,ADI,"NDX, S&P 500",115.38B,2.14B,10.46B,71.03,4.76,3.66 (1.57%),3.50 (1.51%),"Jun 04, 2024",...,27.28%,34.24%,17.10%,1.12,5.83,3.45% 2.14%,251.60,234.56,232.51,-0.87%
4,ADP,"NDX, S&P 500",101.87B,3.70B,18.91B,11.29,8.04,5.16 (2.07%),5.30 (2.13%),"Jun 14, 2024",...,7.68%,16.31%,6.84%,0.80,3.54,1.19% 1.45%,260.63,251.49,248.90,-1.03%
5,TSLA,"NDX, S&P 500",571.63B,13.65B,94.75B,20.19,8.54,-,-,-,...,-23.47%,-2.00%,-27.87%,2.32,7.91,4.25% 3.90%,183.03,173.74,179.24,3.17%
6,MSFT,"DJIA, NDX, S&P 500",3197.08B,86.18B,236.58B,34.06,10.77,2.95 (0.69%),2.93 (0.68%),"May 15, 2024",...,13.84%,37.06%,14.39%,0.90,7.10,1.65% 1.63%,481.64,427.00,430.16,0.74%
7,XOM,S&P 500,508.79B,32.80B,331.92B,52.05,7.43,3.83 (3.38%),3.76 (3.32%),"May 14, 2024",...,9.05%,5.42%,13.44%,0.90,2.18,2.04% 1.78%,132.91,113.51,113.42,-0.08%
8,MELI,NDX,85.86B,1.13B,15.77B,66.87,147.71,-,-,"Dec 28, 2017",...,11.05%,31.88%,7.76%,1.59,47.00,2.53% 2.85%,2017.22,1707.40,1693.57,-0.81%


In [17]:
 data_nasdaq.shape,data_nasdaq.columns

((9, 79),
 Index(['symbol', 'Index', 'Market Cap', 'Income', 'Sales', 'Book/sh',
        'Cash/sh', 'Dividend Est.', 'Dividend TTM', 'Dividend Ex-Date',
        'Employees', 'Option/Short', 'Sales Surprise', 'SMA20', 'P/E',
        'Forward P/E', 'PEG', 'P/S', 'P/B', 'P/C', 'P/FCF', 'Quick Ratio',
        'Current Ratio', 'Debt/Eq', 'LT Debt/Eq', 'EPS Surprise', 'SMA50',
        'EPS (ttm)', 'EPS next Y', 'EPS next Q', 'EPS this Y', 'EPS next Y',
        'EPS next 5Y', 'EPS past 5Y', 'Sales past 5Y', 'EPS Y/Y TTM',
        'Sales Y/Y TTM', 'EPS Q/Q', 'Sales Q/Q', 'SMA200', 'Insider Own',
        'Insider Trans', 'Inst Own', 'Inst Trans', 'ROA', 'ROE', 'ROI',
        'Gross Margin', 'Oper. Margin', 'Profit Margin', 'Payout', 'Earnings',
        'Trades', 'Shs Outstand', 'Shs Float', 'Short Float', 'Short Ratio',
        'Short Interest', '52W Range', '52W High', '52W Low', 'RSI (14)',
        'Recom', 'Rel Volume', 'Avg Volume', 'Volume', 'Perf Week',
        'Perf Month', 'Perf Quarter

In [87]:
datos = data_nasdaq.copy()


#------------------------------------------------------------------------------#
# RECONOCER LOS VALORES AUSENTES

# reemplaza los valores '-', '0', '0.0%' por nan
datos= datos.replace(['-', '0', '0.0%'], np.nan)


#------------------------------------------------------------------------------#
# MANEJO DE COLUMNA INDIVIDUALES

# Dividir la columna 'Volatility'
datos[['Volatility Week', 'Volatility Month']] = datos['Volatility'].str.split(expand=True)

# Dividir la columna '52W Range' en dos nuevas columnas
datos[['52W Range min', '52W Range max']] = datos['52W Range'].str.split(' - ', expand=True)

# Dividir la columna 'Option/Short' en dos nuevas columnas
datos[['Option', 'Short']] = datos['Option/Short'].str.split(' / ', expand=True)

# Convertir la columna 'Volume' a tipo numérico, eliminando comas y saltando errores
datos['Volume'] = pd.to_numeric(datos['Volume'].str.replace(',', '', regex=False), errors='coerce')

# Expresión regular para separar el valor numérico del porcentaje
pattern = r'([\d.]+) \(([\d.]+)%\)'

# Dividir la columna 'Dividend TTM'
datos[['Dividend TTM', 'Dividend TTM(%)']] = datos['Dividend TTM'].str.extract(pattern)
# Dividir la columna 'Dividend Ex-Date'
datos[['Dividend Est.', 'Dividend Est.(%)']] = datos['Dividend Est.'].str.extract(pattern)


# Eliminar las columnas originales
datos.drop(columns=['52W Range','Volatility','Option/Short'], inplace=True)

#------------------------------------------------------------------------------#
#Convertir a numeros los valores con notacion
columnas_a_convertir = ['Market Cap', 'Income', 'Sales','Short Interest','Shs Outstand','Shs Float','Avg Volume']

def convertir_valores(val):
    """
    Convierte un valor de string con sufijo 'B', 'M', 'K' a su valor numérico correspondiente.
    """
    if 'B' in val:
        return float(val.replace('B', '')) * 1_000_000_000
    elif 'M' in val:
        return float(val.replace('M', '')) * 1_000_000
    elif 'K' in val:
        return float(val.replace('K', '')) * 1_000
    else:
        return float(val)

def convertir_columnas(dataframe, columnas):
    """
    Convierte las columnas especificadas de un dataframe utilizando la función convertir_valores.
    """
    for columna in columnas:
        if columna in dataframe.columns:
            dataframe[columna] = dataframe[columna].apply(convertir_valores)
    return dataframe

# Convertir las columnas en el dataframe
datos = convertir_columnas(datos, columnas_a_convertir)


#------------------------------------------------------------------------------#
# Manejo de las columnas que estan expresadas en %
def procesar_porcentajes(dataframe, columnas_exceptuadas):
    """
    Procesa las columnas del dataframe para eliminar el carácter '%' y convertir los valores a numéricos.
    Además, renombra las columnas afectadas para indicar que los valores están en porcentaje.
    """
    for columna in dataframe.columns:
        try:
            dataframe[columna] = dataframe[columna].astype(float)
        
        except ValueError:
            #
            
            if columna not in columnas_exceptuadas:
                # Convertir valores de porcentaje a numéricos
                if dataframe[columna].dtype == 'object' and dataframe[columna].str.contains('%').any():
                    dataframe[columna] = dataframe[columna].str.replace('%', '').astype(float)
                    dataframe.rename(columns={columna: columna + ' (%)'}, inplace=True)

    return dataframe


# Columnas a exceptuar del procesamiento
columnas_a_exceptuar = ['symbol', 'Index', 'Option','Short', 'Earnings', 'Dividend Ex-Date']
# Procesar el dataframe exceptuando las columnas especificadas
datos = procesar_porcentajes(datos, columnas_a_exceptuar)

In [88]:
datos.shape, datos.columns

((9, 84),
 Index(['symbol', 'Index', 'Market Cap', 'Income', 'Sales', 'Book/sh',
        'Cash/sh', 'Dividend Est.', 'Dividend TTM', 'Dividend Ex-Date',
        'Employees', 'Sales Surprise (%)', 'SMA20 (%)', 'P/E', 'Forward P/E',
        'PEG', 'P/S', 'P/B', 'P/C', 'P/FCF', 'Quick Ratio', 'Current Ratio',
        'Debt/Eq', 'LT Debt/Eq', 'EPS Surprise (%)', 'SMA50 (%)', 'EPS (ttm)',
        'EPS next Y', 'EPS next Q', 'EPS this Y (%)', 'EPS next Y (%) (%)',
        'EPS next 5Y (%)', 'EPS past 5Y (%)', 'Sales past 5Y (%)',
        'EPS Y/Y TTM (%)', 'Sales Y/Y TTM (%)', 'EPS Q/Q (%)', 'Sales Q/Q (%)',
        'SMA200 (%)', 'Insider Own (%)', 'Insider Trans (%)', 'Inst Own (%)',
        'Inst Trans (%)', 'ROA (%)', 'ROE (%)', 'ROI (%)', 'Gross Margin (%)',
        'Oper. Margin (%)', 'Profit Margin (%)', 'Payout (%)', 'Earnings',
        'Trades', 'Shs Outstand', 'Shs Float', 'Short Float (%)', 'Short Ratio',
        'Short Interest', '52W High (%)', '52W Low (%)', 'RSI (14)', 'Recom',

In [89]:
# Realizar el merge con los datos de la pagina oficial

merged_df = pd.merge(datos,nasdaq_total2, on='symbol', how='inner')
# Completar los NaN en 'Market Cap' con los valores de 'Market Cap2'
merged_df['Market Cap'].fillna(merged_df['Market Cap2'], inplace=True)

# Eliminar la columna 'Market Cap2'
merged_df.drop(columns='Market Cap2', inplace=True)

In [90]:
merged_df.shape,merged_df.columns

((9, 89),
 Index(['symbol', 'Index', 'Market Cap', 'Income', 'Sales', 'Book/sh',
        'Cash/sh', 'Dividend Est.', 'Dividend TTM', 'Dividend Ex-Date',
        'Employees', 'Sales Surprise (%)', 'SMA20 (%)', 'P/E', 'Forward P/E',
        'PEG', 'P/S', 'P/B', 'P/C', 'P/FCF', 'Quick Ratio', 'Current Ratio',
        'Debt/Eq', 'LT Debt/Eq', 'EPS Surprise (%)', 'SMA50 (%)', 'EPS (ttm)',
        'EPS next Y', 'EPS next Q', 'EPS this Y (%)', 'EPS next Y (%) (%)',
        'EPS next 5Y (%)', 'EPS past 5Y (%)', 'Sales past 5Y (%)',
        'EPS Y/Y TTM (%)', 'Sales Y/Y TTM (%)', 'EPS Q/Q (%)', 'Sales Q/Q (%)',
        'SMA200 (%)', 'Insider Own (%)', 'Insider Trans (%)', 'Inst Own (%)',
        'Inst Trans (%)', 'ROA (%)', 'ROE (%)', 'ROI (%)', 'Gross Margin (%)',
        'Oper. Margin (%)', 'Profit Margin (%)', 'Payout (%)', 'Earnings',
        'Trades', 'Shs Outstand', 'Shs Float', 'Short Float (%)', 'Short Ratio',
        'Short Interest', '52W High (%)', '52W Low (%)', 'RSI (14)', 'Recom',

In [92]:
# Reordenar los inidcadores
orden=['symbol','Index','Name','Country','IPO Year','Sector','Industry','Employees',
'Market Cap','Income','Sales','Book/sh','Cash/sh','Dividend Est.','Dividend TTM',
'Dividend TTM(%)','Dividend Est.(%)','Dividend Ex-Date','Sales Surprise (%)','P/E',
'Forward P/E','PEG','P/S','P/B','P/C','P/FCF','Quick Ratio','Current Ratio',
'Debt/Eq','LT Debt/Eq','EPS Surprise (%)','EPS (ttm)','EPS next Y','EPS next Q',
'EPS this Y (%)','EPS next 5Y (%)','EPS past 5Y (%)','Sales past 5Y (%)',
'EPS Y/Y TTM (%)','EPS Q/Q (%)','Sales Y/Y TTM (%)','Sales Q/Q (%)','Option','Short',
'Insider Own (%)','Insider Trans (%)','Inst Own (%)','Inst Trans (%)','ROA (%)',
'ROE (%)','ROI (%)','Gross Margin (%)','Oper. Margin (%)','Profit Margin (%)','Payout (%)',
'Earnings','Trades','Shs Outstand','Shs Float','Short Float (%)','Short Ratio','Short Interest',
'52W Range min','52W Range max','52W High (%)','52W Low (%)','SMA20 (%)','SMA50 (%)','SMA200 (%)',
'RSI (14)','Rel Volume','Avg Volume','Volume','Perf Week (%)','Perf Month (%)','Perf Quarter (%)',
'Perf Half Y (%)','Perf Year (%)','Perf YTD (%)','Beta','ATR (14)','Volatility Week (%)', 'Volatility Month (%)',
'Recom','Target Price','Prev Close','Price','Change (%)']

# Reorganizar las columnas del DataFrame según el orden especificado
merged_df = merged_df[orden]
merged_df.shape,merged_df.columns

In [91]:
tipos_datos = merged_df.dtypes
conteo_tipos = tipos_datos.value_counts()
conteo_tipos

float64    78
object     11
dtype: int64

In [81]:
# Guardar el DataFrame en un archivo CSV
merged_df.to_csv('datos_Nasdaq.csv', index=False, float_format='%.2f', decimal='.')

# Guardar el DataFrame en un archivo xlsx
merged_df.to_excel('datos_Nasdaq.xlsx',index=False)